#### Get labels from Heustistics
In this file, we use Taxonomy v1.3 and use it to get labels for the images. We save the dataframe containing all the image-label correspondencies in /data folder (not trackable).

After generating the OUTPUT_FILE, we go on and pre-process it and split it in CleanAndSplitData.ipynb.

In [1]:
OUTPUT_FILE = 'data/commonswiki-221218-files-hierarchical-labels.json.bz2'

In [2]:
import pandas as pd
import sys
sys.path.append('../')
from taxonomy.heuristics import Heuristics
from config import *
from tqdm import tqdm

tqdm.pandas()

In [3]:
heuristics = Heuristics()
heuristics.load_graph(HGRAPH_PATH)
heuristics.set_taxonomy(taxonomy_version="v1.3", hierarchical=True)
heuristics.set_heuristics(heuristics_version="headJ+depth")

In [4]:
files = pd.read_parquet(FILES_PATH)

In [5]:
print(files.shape)
files.head(5)

(3891446, 4)


,id,title,url,categories
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu..."
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]"
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ..."
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]"
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint..."


In [6]:
files['labels'] = files.progress_apply(lambda x: heuristics.queryFile(x), axis=1)

100%|██████████| 3891446/3891446 [06:38<00:00, 9765.49it/s] 


In [7]:
files.head()

,id,title,url,categories,labels
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu...","({Culture, Art}, None)"
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]","({}, None)"
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ...","({People, Society, Art, Places, Culture}, None)"
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]","({People, Society}, None)"
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint...","({Culture, Art}, None)"


For some reason, the labels are a tuple with "None" as the second element. Fix that.

In [8]:
files['labels'] = files.progress_apply(lambda x: x.labels[0], axis=1)

100%|██████████| 3891446/3891446 [01:09<00:00, 55689.87it/s]


In [9]:
files.head()

,id,title,url,categories,labels
0,80902489,"""A Gathering of Court Women"", Folio from the D...",c/c1/%22A_Gathering_of_Court_Women%22%2C_Folio...,"[Department of Islamic Art, Metropolitan Museu...","{Culture, Art}"
1,53631642,"""Akdeniz"" - Aliağa, 2016.jpg",f/f8/%22Akdeniz%22_-_Alia%C4%9Fa%2C_2016.jpg,"[Akdeniz (ship, 1955), Aliağa ship-breaking yard]",{}
2,74785910,"""Carnevale di Torino"". Felice Cerruti - f.lli ...",1/13/%22Carnevale_di_Torino%22._Felice_Cerruti...,"[Felice Cerruti Bauduc, Victor Emmanuel II of ...","{People, Society, Art, Places, Culture}"
3,14890941,"""Escena de playa con figura"".jpg",c/c2/%22Escena_de_playa_con_figura%22.jpg,"[Pedro Lira, People with dogs in art]","{People, Society}"
4,17828296,"""Florero con plátanos, limones y libros"", Juan...",d/da/%22Florero_con_pl%C3%A1tanos%2C_limones_y...,"[Works by Juan de Echevarría, Still-life paint...","{Culture, Art}"


In [10]:
files.shape

(3891446, 5)

In [11]:
files.to_json(OUTPUT_FILE)